# Example #1: Exploring Recent Shootings in Philadelphia with Panel, Altair, and Hvplot

In this notebook, we create an example dashboard visualizing the geographic distribution of recent shootings in Philadelphia. The dashboard includes:

- an Altair bar chart of the top 20 neighborhoods with the most shootings
- an hvplot choropleth map showing the total number of shootings per neighborhood
- a slider for selecting the number of days to query shootings for

#### References

For more information, see the documentation:

- [Documentation homepage](https://panel.holoviz.org)
- [User Guide](https://panel.holoviz.org/user_guide/index.html)
    - An overview of the concepts powering Panel dashboards
- [App Gallery](https://panel.holoviz.org/gallery/index.html)
- Examples of end-to-end apps using Panel
- [Reference Gallery](https://panel.holoviz.org/reference/index.html)
    - Examples (code snippets) for the many different kinds of components possible in Panel dashboards
- [Awesome Panel](https://github.com/MarcSkovMadsen/awesome-panel)
    - Github repository of resources and information on Panel
- [Example dashboards from the HoloViz team](https://github.com/holoviz-demos)
- [Folium documentation](https://python-visualization.github.io/folium/)

In [12]:
## NOTE: this has to be first!
import panel as pn

# # Enable Altair!
pn.extension('vega')

In [13]:
import numpy as np
import pandas as pd
import geopandas as gpd
import carto2gpd

import holoviews as hv
import hvplot.pandas

import param as pm

import altair as alt
import folium
from folium.plugins import HeatMap

In [14]:
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

# Load the full data set

We'll load the full data set before initializing the app, and then the user will be able to filter and display subsets of the entire data set. 

The maximum of our slider will be 365 days, so we only need to query data from the past year. When the user slides the slider, we can select the subset of this full dataset to apply.

In [15]:
# maximum number of days into past to query
# this will be the maximum number of days the user will be allowed to select
MAX_DAYS = 365
MIN_DAYS = 30

In [16]:
# Query the CARTO database for shootings
URL = "https://phl.carto.com/api/v2/sql"
WHERE = f"date_ >= current_date - {MAX_DAYS}"
DATA = carto2gpd.get(URL, "shootings", where=WHERE)

# Remove entries with missing values
DATA = DATA.loc[DATA.geometry.notnull()].to_crs(epsg=3857)

# Parse the "date_" column into a Datetime object
DATA["date_"] = pd.to_datetime(DATA["date_"].str.slice(0, 10), format="%Y-%m-%d")

Add neighborhood information as well:

In [17]:
# Read in the neighborhood file
URL = "https://raw.githubusercontent.com/MUSA-550-Fall-2020/week-13/master/data/zillow_neighborhoods.geojson"
HOODS = gpd.read_file(URL).rename(columns={"ZillowName": "Neighborhood"}).to_crs(epsg=3857)

# Perform a spatial join to add a neighborhood column
DATA = gpd.sjoin(DATA, HOODS, how="left", op="within").dropna(subset=["Neighborhood"])

In [18]:
class ShootingsByNeighborhood(pm.Parameterized):
    """
    An example app visualizing recent shootings in Philadelphia. 
    
    It includes:
    - a slider widget
    - an Altair bar chart
    - an hvplot line chart
    
    There is a single parameter:
    
    1. "days": the number of days to query shootings for
    """

    # the number of days to get data for
    days = pm.Integer(default=90, bounds=(MIN_DAYS, MAX_DAYS))

    def filter_by_days(self):
        """
        Return the subset of the full data set ('DATA') that 
        occurred in the last 'self.days' days.
        """
        # Today's date
        today = pd.to_datetime("today")

        # Difference between shootings and today
        diff = today - DATA["date_"]

        # Valid selection: less than X days ago
        selection = diff.dt.days < self.days

        # only return subset of data that is necessary
        subset = DATA.loc[selection]

        return subset

    def get_shootings_by_neighborhood(self):
        """
        Return a DataFrame with two columns ('date_', 'count')
        that gives the total number of shootings in a day.
        """
        # data from past X days
        df = self.filter_by_days()

        # Add the neighborhood geometries
        N = HOODS.merge(
            df.groupby("Neighborhood").size().reset_index(name="Shootings"),
            on="Neighborhood",
            how="left",
        )

        # NaN shootings means no shootings occurred
        N["Shootings"] = N["Shootings"].fillna(0)
        
        return N

    @pm.depends("days")
    def summary_text(self):
        """
        Get a summary of the number of shootings/homicides.
        
        Returns an HTML <p> tag.
        """
        # only filter this by days
        data = self.filter_by_days()

        # count shootings and homicides
        shootings = len(data)
        t = f"<p style='font-size: 20px'>There have been {shootings:,} shootings in the last {self.days} days.</p>"

        return pn.Pane(t, width=500)

    @pm.depends("days")
    def choropleth(self):
        """
        Return an altair histogram of the number of currently selected
        shootings by age.
        """
        # get the filtered data
        data = self.get_shootings_by_neighborhood()

        return data.hvplot.polygons(
            c="Shootings",
            cmap="viridis",
            hover_cols=["Neighborhood", "Shootings"],
            frame_width=400,
            frame_height=375,
            geo=True, 
            crs=3857
        )

    @pm.depends("days")
    def bar_chart(self):
        """
        Return an altair histogram of the number of currently selected
        shootings by age.
        """
        # get the filtered data
        data = self.get_shootings_by_neighborhood()

        # Sort in descending order and select the top 20
        data = data.sort_values(by="Shootings", ascending=False)
        data = data.iloc[:20]

        # create the chart
        chart = (
            alt.Chart(data[["Shootings", "Neighborhood"]])
            .mark_bar()
            .encode(
                x="Shootings",
                y=alt.Y(
                    "Neighborhood:N",
                    sort=alt.EncodingSortField(
                        field="Shootings",  # The field to use for the sort
                        order="descending",  # The order to sort in
                    ),
                ),
                tooltip=["Shootings", "Neighborhood"],
            )
            .properties(width=300, height=500, title="Top 20 Neighborhoods")
        )
        return chart

Initialize the app:

In [19]:
app = ShootingsByNeighborhood(name="")

## Layout our Panel object

We can use a combination of the `Column()` and `Row()` objects in Panel to create the layout in the main component.

#### Notes

- The `app.param` is an automatically generated set of widgets that corresponds to our `param` parameters
- The charts are specified as the functions of our main application, e.g., app.bar_chart is the function that will return our bar chart.

In [20]:
# The title
title = pn.Pane("<h1>Recent Shootings in Philadelphia by Neighborhood</h1>", width=1000)

In [21]:
# Layout the dashboard
panel = pn.Column(
    pn.Row(title),
    pn.Row(app.summary_text, pn.Param(app.param, width=300)),
    pn.Row(app.bar_chart, app.choropleth),
)

### Call servable() and render our Panel object

- The final step is to call the `.servable()` function
- This will render the dashboard directly in the notebook
- It also enables the notebook to be served from `localhost`.

From the command line, we will run:

```
panel serve --show app1.ipynb
```

And see the app live at: `http://localhost:5006/app1`

In [22]:
panel.servable()

Column
    [0] Row
        [0] Markdown(str, width=1000)
    [1] Row
        [0] ParamMethod(method)
        [1] Param(ShootingsByNeighborhood, width=300)
    [2] Row
        [0] ParamMethod(method)
        [1] ParamMethod(method)